##### https://github.com/TimoBolkart/TF_FLAME

In [1]:
# !pip install git+https://github.com/MPI-IS/mesh.git

In [2]:
# !pip install chumpy

In [3]:
# https://github.com/TimoBolkart/TF_FLAME/blob/master/requirements.txt
import os
import cv2
import numpy as np
import tensorflow as tf # tf==1.15.2/ tf-gpu==1.15.2
from utils.landmarks import load_embedding, tf_get_model_lmks, tf_project_points
from tf_smpl.batch_smpl import SMPL
from tensorflow.contrib.opt import ScipyOptimizerInterface as scipy_pt

In [4]:
def fit_lmk2d(target_img, target_2d_lmks, model_fname, lmk_face_idx, lmk_b_coords, weights):
    '''
    Fit FLAME to 2D landmarks
    :param target_2d_lmks      target 2D landmarks provided as (num_lmks x 3) matrix
    :param model_fname         saved FLAME model
    :param lmk_face_idx        face indices of the landmark embedding in the FLAME topology
    :param lmk_b_coords        barycentric coordinates of the landmark embedding in the FLAME topology
                                (i.e. weighting of the three vertices for the trinagle, the landmark is embedded in
    :param weights             weights of the individual objective functions
    :param visualize           visualize fitting progress
    :return: a mesh with the fitting results
    '''

    tf_trans = tf.Variable(np.zeros((1,3)), name="trans", dtype=tf.float64, trainable=True)
    tf_rot = tf.Variable(np.zeros((1,3)), name="rot", dtype=tf.float64, trainable=True)
    tf_pose = tf.Variable(np.zeros((1,12)), name="pose", dtype=tf.float64, trainable=True)
    tf_shape = tf.Variable(np.zeros((1,300)), name="shape", dtype=tf.float64, trainable=True)
    tf_exp = tf.Variable(np.zeros((1,100)), name="expression", dtype=tf.float64, trainable=True)
    smpl = SMPL(model_fname)
    tf_model = tf.squeeze(smpl(tf_trans,
                               tf.concat((tf_shape, tf_exp), axis=-1),
                               tf.concat((tf_rot, tf_pose), axis=-1)))

    with tf.Session() as session:
        # session.run(tf.global_variables_initializer())

        # Mirror landmark y-coordinates
        target_2d_lmks[:,1] = target_img.shape[0]-target_2d_lmks[:,1]

        lmks_3d = tf_get_model_lmks(tf_model, smpl.f, lmk_face_idx, lmk_b_coords)

        s2d = np.mean(np.linalg.norm(target_2d_lmks-np.mean(target_2d_lmks, axis=0), axis=1))
        s3d = tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.square(lmks_3d-tf.reduce_mean(lmks_3d, axis=0))[:, :2], axis=1)))
        tf_scale = tf.Variable(s2d/s3d, dtype=lmks_3d.dtype)

        # trans = 0.5*np.array((target_img.shape[0], target_img.shape[1]))/tf_scale
        # trans = 0.5 * s3d * np.array((target_img.shape[0], target_img.shape[1])) / s2d
        lmks_proj_2d = tf_project_points(lmks_3d, tf_scale, np.zeros(2))

        factor = max(max(target_2d_lmks[:,0]) - min(target_2d_lmks[:,0]),max(target_2d_lmks[:,1]) - min(target_2d_lmks[:,1]))
        lmk_dist = weights['lmk']*tf.reduce_sum(tf.square(tf.subtract(lmks_proj_2d, target_2d_lmks))) / (factor ** 2)
        neck_pose_reg = weights['neck_pose']*tf.reduce_sum(tf.square(tf_pose[:,:3]))
        jaw_pose_reg = weights['jaw_pose']*tf.reduce_sum(tf.square(tf_pose[:,3:6]))
        eyeballs_pose_reg = weights['eyeballs_pose']*tf.reduce_sum(tf.square(tf_pose[:,6:]))
        shape_reg = weights['shape']*tf.reduce_sum(tf.square(tf_shape))
        exp_reg = weights['expr']*tf.reduce_sum(tf.square(tf_exp))

        session.run(tf.global_variables_initializer())

        def on_step(*_):
            # print(tf_exp.numpy())
            pass

        print('Optimize rigid transformation')
        vars = [tf_scale, tf_trans, tf_rot]
        loss = lmk_dist
        optimizer = scipy_pt(loss=loss, var_list=vars, method='L-BFGS-B', options={'disp': 1, 'ftol': 5e-6})
        optimizer.minimize(session, fetches=[tf_model, tf_scale, tf.constant(smpl.f), tf.constant(target_img), tf.constant(target_2d_lmks), lmks_proj_2d], loss_callback=on_step)

        print('Optimize model parameters')
        vars = [tf_scale, tf_trans[:2], tf_rot, tf_pose, tf_shape, tf_exp]
        loss = lmk_dist + shape_reg + exp_reg + neck_pose_reg + jaw_pose_reg + eyeballs_pose_reg
        optimizer = scipy_pt(loss=loss, var_list=vars, method='L-BFGS-B', options={'disp': 0, 'ftol': 1e-7})
        optimizer.minimize(session, fetches=[tf_model, tf_scale, tf.constant(smpl.f), tf.constant(target_img), tf.constant(target_2d_lmks), lmks_proj_2d,
                                             lmk_dist, shape_reg, exp_reg, neck_pose_reg, jaw_pose_reg, eyeballs_pose_reg], loss_callback=on_step)

        print('Fitting done')
        np_verts, np_scale, np_shape, np_exp, np_trans, np_rot, np_pose = session.run([tf_model, tf_scale, tf_shape, tf_exp, tf_trans, tf_rot, tf_pose])
        
        # for interactive viewwer
        print("First 10 shape values:")
        print(np.round(np_shape[0][:10], 2))
        print("First 10 expression values:")
        print(np.round(np_exp[0][:10], 2))
        
        return np_verts, np_scale, np_shape, np_exp, np_trans, np_rot, np_pose

In [5]:
def run_2d_lmk_fitting(model_fname, flame_lmk_path, target_img_path, target_lmk_path, out_path):
    if not os.path.exists(out_path):
        os.makedirs(out_path)

    lmk_face_idx, lmk_b_coords = load_embedding(flame_lmk_path)
    target_img = cv2.imread(target_img_path)
    lmk_2d = np.load(target_lmk_path)

    weights = {}
    # Weight of the landmark distance term
    weights['lmk'] = 1.0
    # Weight of the shape regularizer
    weights['shape'] = 1e-3
    # Weight of the expression regularizer
    weights['expr'] = 1e-3
    # Weight of the neck pose (i.e. neck rotationh around the neck) regularizer
    weights['neck_pose'] = 100.0
    # Weight of the jaw pose (i.e. jaw rotation for opening the mouth) regularizer
    weights['jaw_pose'] = 1e-3
    # Weight of the eyeball pose (i.e. eyeball rotations) regularizer
    weights['eyeballs_pose'] = 10.0

    # result_mesh, result_scale = fit_lmk2d(target_img, lmk_2d, model_fname, lmk_face_idx, lmk_b_coords, weights)
    np_verts, np_scale, np_shape, np_exp, np_trans, np_rot, np_pose = fit_lmk2d(target_img, lmk_2d, model_fname, lmk_face_idx, lmk_b_coords, weights)
    
    image_basename = os.path.splitext(os.path.basename(target_img_path))[0]
    model_basename = os.path.splitext(os.path.basename(model_fname))[0]
    num_lmk = str(len(lmk_face_idx))
    basename = image_basename + "_" + model_basename + "_" + num_lmk
    basepath = os.path.join(out_path, basename)
    
    # save required
    np.save(basepath + "_verts", np_verts)
    np.save(basepath + "_scale", np_scale)
    np.save(basepath + "_shape", np_shape)
    np.save(basepath + "_exp", np_exp)
    np.save(basepath + "_trans", np_trans)
    np.save(basepath + "_rot", np_rot)
    np.save(basepath + "_pose", np_pose)

In [6]:
out_path = "./data"

In [9]:
model_fname = "./generic_model.pkl"
target_img_path = "./data/imgHQ01148.jpeg"
target_lmk_path = "./data/imgHQ01148_lmks_68.npy"

In [10]:
if target_lmk_path[-6:-4] == "51":
    flame_lmk_path = "./flame_static_embedding.pkl"
elif target_lmk_path[-6:-4] == "68":
    flame_lmk_path = "./flame_static_embedding_68.pkl"
else:
    flame_lmk_path = "./flame_static_embedding.pkl"
run_2d_lmk_fitting(model_fname, flame_lmk_path, target_img_path, target_lmk_path, out_path)

Optimize rigid transformation
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.027159
  Number of iterations: 16
  Number of functions evaluations: 21
Optimize model parameters
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.010990
  Number of iterations: 151
  Number of functions evaluations: 168
Fitting done
First 10 shape values:
[ 0.56 -0.1   0.2  -0.49  0.07  0.21 -0.34 -0.01  0.05  0.31]
First 10 expression values:
[ 0.3  -0.04  0.03  0.15  0.22  0.33 -0.11 -0.13 -0.16  0.37]
